In [2]:
# Cell 1: Imports and Abstract Base Classes

from abc import ABC, abstractmethod


In [27]:
# Cell 2: Domain Classes

class Soil:
    def __init__(self, soil_type):
        self.soil_type = soil_type.lower()
        # Define common soil characteristics
        soil_characteristics = {
            "loamy": {"nutrient_retention": "high", "pH_range": (6.0, 7.5)},
            "sandy": {"nutrient_retention": "low", "pH_range": (5.5, 6.5)},
            "clay": {"nutrient_retention": "medium", "pH_range": (6.0, 7.0)},
            "peat": {"nutrient_retention": "high", "pH_range": (5.5, 6.5)},
            "alluvium": {"nutrient_retention": "high", "pH_range": (6.0, 7.0)}
        }
        self.attributes = soil_characteristics.get(self.soil_type,
                                                     {"nutrient_retention": "medium", "pH_range": (6.0, 7.0)})



In [28]:
class Environment:
    def __init__(self, temperature, humidity):
        self.temperature = temperature  # in °C
        self.humidity = humidity        # in %
        # Optimal conditions per plant type
        self.optimal_conditions = {
            "banana": {"temperature": (25, 30), "humidity": (50, 80)},
            "coffee": {"temperature": (17, 26), "humidity": (65, 85)},
            "rice":   {"temperature": (20, 30), "humidity": (70, 90)}
        }


In [30]:
class Plant(ABC):
    def __init__(self, name):
        self.name = name.lower()
        self.nutrients = {}  # To be defined in subclasses
        self.domain_info = {}  # Contains domain-specific guidelines

    @abstractmethod
    def set_nutrients(self):
        pass

In [17]:
class Banana(Plant):
    def __init__(self):
        super().__init__("banana")
        self.domain_info = {
            "optimal_soil": "loamy",
            "pH_range": (6.0, 7.5),
            "nutrients": {
                "potassium": "Foliar spray of 2% KCl weekly until symptoms disappear.",
                "iron": "Soil application of FeSO4 (5 g/ha) or foliar spray of 0.5% FeSO4 weekly.",
                "magnesium": "Foliar spray of 5% MgSO4 or application of dolomite limestone at 3 t/ha."
            }
        }
        self.set_nutrients()

    def set_nutrients(self):
        self.nutrients = {"potassium": None, "iron": None, "magnesium": None}


In [31]:
# Coffee: requires potassium, iron, and magnesium
class Coffee(Plant):
    def __init__(self):
        super().__init__("coffee")
        self.domain_info = {
            "optimal_soil": "peat-based, slightly acidic",
            "pH_range": (5.5, 6.5),
            "nutrients": {
                "potassium": "Foliar application of 0.1% MOP.",
                "iron": "Ensure good drainage; use iron sulphate or foliar spray of iron chelate.",
                "magnesium": "Soil application of dolomitic lime or foliar spray of 0.1% magnesium sulphate."
            }
        }
        self.set_nutrients()

    def set_nutrients(self):
        self.nutrients = {"potassium": None, "iron": None, "magnesium": None}


In [32]:
# Rice: requires nitrogen, phosphorus, and potassium
class Rice(Plant):
    def __init__(self):
        super().__init__("rice")
        self.domain_info = {
            "optimal_soil": "clay or clay loam with good water retention",
            "pH_range": (5.5, 6.5),
            "nutrients": {
                "nitrogen": "Apply nitrate-based fertilizers in split doses for healthy foliage.",
                "phosphorus": "Basal application to promote root development and tillering.",
                "potassium": "Maintain adequate levels to improve spikelet formation and grain filling."
            }
        }
        self.set_nutrients()

    def set_nutrients(self):
        self.nutrients = {"nitrogen": None, "phosphorus": None, "potassium": None}

In [34]:
# Cell 4: Deficiency Evaluator

class DeficiencyEvaluator:
    """
    Evaluates nutrient deficiency severity based on soil nutrient retention.
    Mapping: high retention -> Mild deficiency; medium -> Moderate; low -> Severe.
    """
    def __init__(self, plant, soil, environment):
        self.plant = plant
        self.soil = soil
        self.environment = environment
        self.severity_mapping = {"high": "Mild", "medium": "Moderate", "low": "Severe"}

    def evaluate(self):
        deficiency_results = {}
        retention = self.soil.attributes.get("nutrient_retention", "medium")
        severity = self.severity_mapping.get(retention, "Moderate")
        # For demonstration, assign the same severity to all nutrients.
        for nutrient in self.plant.nutrients.keys():
            deficiency_results[nutrient] = severity
        return deficiency_results


In [35]:
# Cell 5: Recommendation Strategies and Recommendation Engine

class RecommendationStrategy(ABC):
    @abstractmethod
    def get_recommendation(self, nutrient, domain_recommendation):
        pass

class MildDeficiencyStrategy(RecommendationStrategy):
    def get_recommendation(self, nutrient, domain_recommendation):
        return f"Mild deficiency for {nutrient}. Standard maintenance is sufficient. ({domain_recommendation})"

class ModerateDeficiencyStrategy(RecommendationStrategy):
    def get_recommendation(self, nutrient, domain_recommendation):
        return f"Moderate deficiency for {nutrient}. Consider moderate fertilizer application. ({domain_recommendation})"

class SevereDeficiencyStrategy(RecommendationStrategy):
    def get_recommendation(self, nutrient, domain_recommendation):
        return f"Severe deficiency for {nutrient}. Intensive correction required. ({domain_recommendation})"

class RecommendationEngine:
    def __init__(self, plant, deficiency_results):
        self.plant = plant
        self.deficiency_results = deficiency_results
        self.strategy_map = {
            "Mild": MildDeficiencyStrategy(),
            "Moderate": ModerateDeficiencyStrategy(),
            "Severe": SevereDeficiencyStrategy()
        }

    def generate_recommendations(self):
        recommendations = {}
        for nutrient, severity in self.deficiency_results.items():
            strategy = self.strategy_map.get(severity, ModerateDeficiencyStrategy())
            domain_rec = self.plant.domain_info["nutrients"].get(nutrient, "No guideline available.")
            recommendations[nutrient] = strategy.get_recommendation(nutrient, domain_rec)
        return recommendations

    def additional_details(self, soil, environment):
        details = {}
        # Soil suitability: Check if the soil type contains the optimal soil keyword.
        optimal_soil = self.plant.domain_info["optimal_soil"].lower()
        details["soil_suitability"] = "Suitable" if optimal_soil in soil.soil_type else "Not optimal"

        # Temperature and humidity evaluation
        optimal = environment.optimal_conditions.get(self.plant.name, {"temperature": (0,0), "humidity": (0,0)})
        temp_status = "matches" if optimal["temperature"][0] <= environment.temperature <= optimal["temperature"][1] else "does not match"
        humidity_status = "matches" if optimal["humidity"][0] <= environment.humidity <= optimal["humidity"][1] else "does not match"
        details["temperature"] = f"Temperature {temp_status} optimal range."
        details["humidity"] = f"Humidity {humidity_status} optimal range."
        return details


In [36]:
# Cell 6: Main Implementation & Testing

def main():
    # Simulated user inputs (can be replaced with input() in interactive sessions)
    soil_input = "loamy"      # Options: loamy, sandy, clay, peat, alluvium, etc.
    temperature_input = 28    # in °C
    humidity_input = 75       # in %
    plant_choice = "banana"   # Options: banana, coffee, rice

    # Create instances
    soil = Soil(soil_input)
    environment = Environment(temperature_input, humidity_input)

    if plant_choice.lower() == "banana":
        plant = Banana()
    elif plant_choice.lower() == "coffee":
        plant = Coffee()
    elif plant_choice.lower() == "rice":
        plant = Rice()
    else:
        print("Invalid plant choice. Defaulting to Banana.")
        plant = Banana()

    # Evaluate nutrient deficiency
    evaluator = DeficiencyEvaluator(plant, soil, environment)
    deficiency_results = evaluator.evaluate()

    # Generate recommendations
    engine = RecommendationEngine(plant, deficiency_results)
    nutrient_recommendations = engine.generate_recommendations()
    extra_details = engine.additional_details(soil, environment)

    # Display results
    print("=== Fertilizer Recommendations ===")
    for nutrient, recommendation in nutrient_recommendations.items():
        print(f"{nutrient.capitalize()}: {recommendation}")

    print("\n=== Additional Details ===")
    for key, detail in extra_details.items():
        print(f"{key.capitalize()}: {detail}")

# Run the main function
if __name__ == "__main__":
    main()


=== Fertilizer Recommendations ===
Potassium: Mild deficiency for potassium. Standard maintenance is sufficient. (Foliar spray of 2% KCl weekly until symptoms disappear.)
Iron: Mild deficiency for iron. Standard maintenance is sufficient. (Soil application of FeSO4 (5 g/ha) or foliar spray of 0.5% FeSO4 weekly.)
Magnesium: Mild deficiency for magnesium. Standard maintenance is sufficient. (Foliar spray of 5% MgSO4 or application of dolomite limestone at 3 t/ha.)

=== Additional Details ===
Soil_suitability: Suitable
Temperature: Temperature matches optimal range.
Humidity: Humidity matches optimal range.
